# Workshop 03: Image Data Augmentation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/ei-workshop-image-data-augmentation/blob/master/workshop_03_image_data_augmentation.ipynb)

Script for creating an augmented image dataset. It will transform input images to create a series of augmented samples that are saved in a new output directory.

The first few cells will download an example dataset (electronic components) and create a folder named "dataset" in the /content directory. The images will be divided into their respective classes, where each class has its own folder with the name of the class. For example:

<pre>
/content
    |- dataset
        |- background
        |- capacitor
        |- diode
        |- led
        |- resistor
</pre>

The original images along with their transforms will be saved in the output directory. Each output file will be the original filename appended with "_{num}" where {num} is some incrementing value based on the total number of transforms performed per image.

For example, if you have a file named "0.png" in /content/dataset/resistor, it will become "0_0.png" in /content/output/resistor. The first transform will be "0_1.png", the second transform will be "0_2.png" and so on.

Run each of the cells paying attention to their contents and output. Fill out the necessary parts of the functions where you find the following comment:

```
# >>> ENTER YOUR CODE HERE <<<
```

Author: EdgeImpulse, Inc.<br>
Modified by: Shawn Hymel<br>
Date: September 22, 2021<br>
License: [Apache-2.0](apache.org/licenses/LICENSE-2.0)<br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import PIL

import skimage.transform
import skimage.util

In [ ]:
### Settings

# Location of dataset and output folder
DATASET_URL = "https://github.com/ShawnHymel/ei-workshop-image-data-augmentation/blob/master/Datasets/electronic-components-png.zip?raw=true"
DOWNLOAD_PATH = "/content/electronic-components-png"
DATASET_PATH = "/content/dataset"
OUT_PATH = "/content/output"
OUT_ZIP = "augmented_dataset.zip"

# File format to use for new dataset
IMG_EXT = ".png"

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)

In [ ]:
### Download and unzip original dataset (use '!' to call Linux commands)
!wget "{DATASET_URL}" -O temp.zip
!unzip -q temp.zip
!rm temp.zip
!mv "{DOWNLOAD_PATH}" "{DATASET_PATH}"

In [ ]:
### Create output directory
try:
  os.makedirs(OUT_PATH)
except FileExistsError:
  print("WARNING: Output directory already exists. Check to make sure it is empty.")

## Transform Functions

Use these functions to create transformed copies of your images. Supply a raw image (Numpy array) along with other arguments to get a list of output image copies. Call these functions to create your augmented dataset

### Flip Images
Create 3 new flipped copies of the input image

In [ ]:
def create_flipped(img):
  """Creates 3 copies of the input image that are flipped along x and y dimensions.

  Parameters
  ----------
  img : ndarray
    3D array containing raw image data (3 color channels) with `int` type.

  Returns
  -------
  list
    A list of ndarray images

  Examples
  --------
  >>> flipped_img_list = create_flipped(img)
  >>> print(len(flipped_img_list))
  3

  """

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped

### Rotate Images

Create a number of rotated copies of the input image. `rotations` should be a list of rotation angles (e.g. 0..360).

In [ ]:
def create_rotated(img, rotations):
  """Creates copies of input image (img) that have been rotated (given by the list `rotations`).

  Parameters
  ----------
  img : ndarray
    3D array containing raw image data (3 color channels) with `int` type.
  rotations : list of float
    List of rotations (counter-clockwise direction).

  Returns
  -------
  list
    A list of ndarray images

  Examples
  --------
  >>> rotated_img_list = create_rotated(img, [45, 90, 135])
  >>> print(len(rotated_img_list))
  3

  """

  # Create list of rotated images (keep 8-bit values)
  rotated = []
  for rot in rotations:
    img_rot = skimage.transform.rotate(img, angle=rot, mode='edge', preserve_range=True)
    img_rot = img_rot.astype(np.uint8)
    rotated.append(img_rot)

  return rotated

### Random Zooms

Create copies of image that has been zoomed and cropped `num_crops` number of times.

In [ ]:
def create_random_zooms(img, scale_factor, num_crops):
  """Creates copies of zoom and random crops of original image.

  Parameters
  ----------
  img : ndarray
    3D array containing raw image data (3 color channels) with `int` type.
  scale_factor : float
    Scale factor of original image (e.g. 1.5 for 1.5x zoom).
  num_crops : int
    Number of random crops to produce after scaling the image.

  Returns
  -------
  list
    A list of ndarray images

  Examples
  --------
  >>> cropped_img_list = create_random_zooms(img, 1.3, 2)
  >>> print(len(cropped_img_list))
  2

  """

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create scaled images (e.g. make the image bigger) and keep 8-bit values
  img_scaled = skimage.transform.rescale(img, 
                                        scale=scale_factor, 
                                        anti_aliasing=True, 
                                        multichannel=True,
                                        preserve_range=True)
  img_scaled = img_scaled.astype(np.uint8)

  # Get height and width of scaled image
  s_h = img_scaled.shape[0]
  s_w = img_scaled.shape[1]

  # Create list of random zooms
  zooms = []
  for i in range(num_crops):
    
    # Randomly choose start of crop point
    crop_y = round(random.random() * (s_h - height))
    crop_x = round(random.random() * (s_w - width))

    # Crop scaled image
    zoom = img_scaled[crop_y:(crop_y + height), crop_x:(crop_x + width), :]

    # Append zoomed image to list
    zooms.append(zoom)

  return zooms

### Random Translations
Create `num_translations` number of copies of input image that have been randomly translated (i.e. moved) up to 25% of original width or hieght.

In [ ]:
def create_random_translations(img, num_translations):
  """Creates copies of random translations of original image.

  Parameters
  ----------
  img : ndarray
    3D array containing raw image data (3 color channels) with `int` type.
  num_translations : int
    Number of random translations to produce.

  Returns
  -------
  list
    A list of ndarray images

  Examples
  --------
  >>> cropped_img_list = create_random_translations(img, 2)
  >>> print(len(cropped_img_list))
  2

  """

  # Get height and width of original image
  height = img.shape[0]
  width = img.shape[1]

  # Create list of random translations
  translations = []
  for i in range(num_translations):
  
    # Choose random amount to translate (up to 1/4 image width, height) in either direction
    tr_y = round((0.5 - random.random()) * (height / 2))
    tr_x = round((0.5 - random.random()) * (width / 2))

    # Perform translation to create new image
    translation = skimage.transform.AffineTransform(translation=(tr_y, tr_x))
    img_tr = skimage.transform.warp(img, translation, mode='edge', preserve_range=True)
    img_tr = img_tr.astype(np.uint8)

    # Append translated image to list
    translations.append(img_tr)

  return translations

### Add Noise
**TO BE IMPLEMENTED BY STUDENT**

In [ ]:
# >>>ENTER YOUR CODE HERE<<<

## Do Transforms

Call your functions to create a set of augmented data.

In [ ]:
### Function to open image and create a list of new transforms
# NOTE: You will need to call your transform functions here!
def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Example: Perform transforms (call your transform functions)
  img_tfs.append(create_flipped(img_array))

  # Call other transform functions (continue to append output to img_tfs)
  # >>>ENTER YOUR CODE HERE<<<

  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs

In [ ]:
### Load all images, create transforms, and save in output directory

# Find the directories in the dataset folder (skip the Jupyter Notebook checkpoints hidden folder)
for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if os.path.isdir(class_dir) and label != ".ipynb_checkpoints":

    # Create output directory
    out_path = os.path.join(OUT_PATH, label)
    os.makedirs(out_path, exist_ok=True)

    # Go through each image in the subfolder
    for i, filename in enumerate(os.listdir(class_dir)):

      # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
      if filename != ".ipynb_checkpoints":

        # Get the root of the filename before the extension
        file_root = os.path.splitext(filename)[0]

        # Do all transforms for that one image
        file_path = os.path.join(DATASET_PATH, label, filename)
        img_tfs = create_transforms(file_path)

        # Save images to new files in output directory
        for i, img in enumerate(img_tfs):

          # Create a Pillow image from the Numpy array
          img_pil = PIL.Image.fromarray(img)

          # Construct filename (<orignal>_<transform_num>.<EXT>)
          out_file_path = os.path.join(out_path, file_root + "_" + str(i) + IMG_EXT)

          # Convert Numpy array to image and save as a file
          img_pil = PIL.Image.fromarray(img)
          img_pil.save(out_file_path)

In [ ]:
### Zip our new dataset (use '!' to call Linux commands)
!zip -r -q "{OUT_ZIP}" "{OUT_PATH}"